In [8]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import urllib.request
from six.moves import cPickle as pickle
import tensorflow as tf
from tensorflow.models.rnn.ptb import reader
from gensim import corpora


from model import  dynamic_RNN_model

In [9]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [10]:
def dynamic_RNN_model(
    num_hid_units = 3,
    vocab_size = 7,
    momentum = 0.9,
    learning_rate = 0.01
    ):
    
    num_classes = vocab_size

    reset_graph()
    
    x = tf.placeholder(tf.int32, shape = [None, None], name='input_placeholder')
    y = tf.placeholder(tf.int32, shape = [None, None], name='output_placeholder')
    batch_size = tf.shape(x)[0]
    

    # ENBEDDING(INPUT) LAYER OPERATION
    # Creating an Embedding matrix with a random weight for all vacab to hidden_matrix
    embed_to_hid_wghts = tf.get_variable('embedding_matrix', [vocab_size, num_hid_units])
    embed_to_hid_layer = tf.nn.embedding_lookup(embed_to_hid_wghts, x)


    # HIDDEN LAYER OPERATION
    rnn_cell = tf.nn.rnn_cell.LSTMCell(num_hid_units, state_is_tuple=True)
    init_state = rnn_cell.zero_state(batch_size, tf.float32)  # Each sequence will hava a state that it passes to its next sequence
    rnn_outputs, new_state = tf.nn.dynamic_rnn(
                                        cell=rnn_cell,
                                        # sequence_length=X_lengths,
                                        initial_state=init_state,
                                        inputs=embed_to_hid_layer)
    

    # OUTPUT LAYER OPERATION
    # Initialize the weight and biases for the output layer. We use variable scope because we would like to share the weights 
    with tf.variable_scope('output_layer'):
        hid_to_output_wght = tf.get_variable('hid_to_output_wght', 
                                                 [num_hid_units, num_classes], 
                                                 initializer = tf.random_normal_initializer())
        output_bias = tf.get_variable('output_bias',
                                      [num_classes],
                                      initializer = tf.random_normal_initializer())
    
    rnn_outputs = tf.reshape(rnn_outputs, [-1, num_hid_units])  
    hid_to_ouptut_layer = tf.matmul(rnn_outputs, hid_to_output_wght) +  output_bias
    output_state = tf.nn.softmax(hid_to_ouptut_layer, name=None)
 
    
    
    # CALCULATING LOSS, OPTIMIZING THE COST FUNCTION, MEASURING ACCURACY
    loss_CE = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(hid_to_ouptut_layer, tf.reshape(y, [-1])))
    
    # The sparse_softmax uses dtype as int32 or int64
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                            momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(loss_CE)
    # y_ = tf.reshape(y, [-1])
    # correct_prediction = tf.equal(tf.arg_max(output_state,1), tf.arg_max(y_ ,1))
    # accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    

    # Returns graph objects
    return dict(
        x=x,
        y=y,
        batch_size = batch_size,
        init_state = init_state,
        new_state = new_state,
        loss_CE = loss_CE,
        optimizer = optimizer,
        training_prediction = output_state
        # accuracy = tf.Variable([[1,2,3]])
    )


In [11]:
class Train():
    def __init__(self):
        self.num_hid_units = 3,
        self.momentum = 0.9,
        self.learning_rate = 0.5
        self.train_batch_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Word-Nets/training_batch/'
        dictionary_dir = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Word-Nets/dictionary.txt'
        self.vocab_size = len(corpora.Dictionary.load_from_text(dictionary_dir))

    def accuracy(self, predictions, labels, labels_one_hot = None):
        # The input labels are a One-Hot Vector
        if labels_one_hot:
            return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
                  / predictions.shape[0])
        else:
            return (100.0 * np.sum(np.argmax(predictions, 1) == np.reshape(labels, [-1]))
                  / predictions.shape[0])
        

    def train_network(self, num_batches, graph_dict):
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            
            aa = np.array([[1,2,3,4,5], [1,3,4,0,5], [2,3,4,5,6]])  # , [2,3,4,5,6], [2,1,5,4,3]
            bb = np.array([[2,3,4,5,2], [3,4,0,5,2], [3,4,5,6,2]])  # , [3,4,5,6,2], [1,5,4,3,2]
            cc = np.array([[1,4,2,2,1,3], [1,4,3,3,0,0]])
            dd = np.array([[4,2,2,5,3,2], [4,3,0,5,3,0]])
            epochs = 1
            for epoch in np.arange(epochs):
                new_hid_layer_state = None
                print (graph_dict)

                for no in np.arange(num_batches):#[[aa,bb],[cc,dd]]:#np.arange(2):
                    with open(self.train_batch_dir+'batch'+str(no)+'.pickle', 'rb') as f:
                        dataset = pickle.load(f)
                        
                        batch_train_dataset = dataset['batch_train_dataset']
                        batch_train_labels = dataset['batch_train_labels']

                        if not new_hid_layer_state: 
                            feed_dict= {graph_dict['x']: batch_train_dataset, 
                                        graph_dict['y']: batch_train_labels}
                                        # graph_dict['batch_size']: batch_size}
                        else:
                            print ('Using the new RNN State')
                            feed_dict= {graph_dict['x']: batch_train_dataset, 
                                        graph_dict['y']: batch_train_labels,
                                        graph_dict['init_state'] : new_hid_layer_state}

                        bs, nwst, loss, opt, tp = sess.run([graph_dict['batch_size'],
                                                        graph_dict['new_state'],
                                                        graph_dict['loss_CE'],
                                                        graph_dict['optimizer'],
                                                        graph_dict['training_prediction']], 
                                                        feed_dict=feed_dict)
                        new_hid_layer_state = nwst

                        acc = self.accuracy(tp, batch_train_labels)

                        # print ('Batch size \n', bs)
                        # print ('')
                        # print ('New State \n', nwst)
                        # print ('')
                        # print ('Avg Loss \n',  loss)
                        # print ('')
                        # print ('optimizer \n', opt)
                        # print ('')
                        # print ('training_prediction \n', tp)
                        # print ('')
                        print ('accuracy \n', acc)
                        print ('')
                        print ('')
                        print ('popopopopopopopoop')
                        print ('')
                        print ('')

In [12]:
obj_Train = Train()
graph_dict =  dynamic_RNN_model(num_hid_units = 3,
                                vocab_size = obj_Train.vocab_size,
                                momentum = 0.9,
                                learning_rate = 0.01)
obj_Train.train_network(num_batches = 2, graph_dict)

SyntaxError: positional argument follows keyword argument (<ipython-input-12-1cf8568353a7>, line 6)